First let's make sure that we import all the required libraries to parse the wikipedia webpage, extract the table from it and populate it into the pandas dataframe.

In [1]:
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json
import ssl
import pandas as pd

Then we need to define the url of the webpage that we will parse, then parse it, extract the table, define column names, rows and prepare a dataframe which we will be populating with the information from the table on the webpage.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
html = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(html,'html.parser')
table = soup.find('table', {'class':'wikitable sortable'}).tbody
rows = table.find_all('tr')
columns = [v.text.replace('\n','') for v in rows[0].find_all('th')]

df = pd.DataFrame(columns = columns)
df_hoods = df

Once the dataframe is ready we need to create a loop which will iterate through the lines of html code that we have extracted using BeautifulSoup. We also need to create an "if" condition to ignore the rows where 'Borough' cell is equl to 'Not assigned'. If it is not then we populate our dataframe with values from the row.

In [5]:
for i in range(1,len(rows)):
    tds = rows[i].find_all('td')
    
    if tds[1].text == 'Not assigned':
        continue
    else:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', '')]
        df_hoods = df_hoods.append(pd.Series(values, index = columns), ignore_index = True)
df_hoods.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Further we check whether 'Not assigned' value still exist in the dataframe in both 'Borough' and 'Neighbourhood' columns.

In [21]:
print('Not assigned' in df_hoods['Borough'].unique())
print('Not assigned' in df_hoods['Neighbourhood'].unique())

False
True


So as it is seen from the results of the previous cell's code there are still 'Not assigned' values in the 'Neighbourhood' column. Therefore, we need to replace those values with the corresponding 'Borough' values. I have created a loop to iterate through values in the 'Neighbourhood' column and if 'Not assigned' is found replaced it with the value in the corresponding 'Borough' column.

In [25]:
for i in range(0,len(df_hoods['Neighbourhood'])):
    if df_hoods['Neighbourhood'].iloc[i] == 'Not assigned':
        df_hoods['Neighbourhood'].iloc[i] = df_hoods['Borough'].iloc[i]
df_hoods.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Now lets check one more time if 'Not assigned' value exists in any of the two columns.

In [26]:
print('Not assigned' in df_hoods['Borough'].unique())
print('Not assigned' in df_hoods['Neighbourhood'].unique())

False
False


Now that we have removed all 'Not assigned' values, there is still one more step to do to clean out the data. As you may have noticed there might be several neighbourhoods in the same postalcode area. We need to group such neighbourhoods into a single row and list all the neighbourhoods in the single cell separated by coma. I have applied groupby attribute of the pandas dataframes along with applying formating to separate neighbourhood names by coma.

In [29]:
df_clean = df_hoods.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_clean.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae"
5,M1J,Scarborough,"Scarborough Village, Scarborough Village"
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park, E..."
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge, Clairlea, Gol..."
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village Wes..."
9,M1N,Scarborough,"Birch Cliff, Cliffside West, Birch Cliff, Clif..."


Finally, as it was requested in the requirements to this assignment we need to check the shape of the clean dataframe that we have obtained from wikipedia.

In [49]:
df_clean.shape

(103, 3)

Thank you! I hope the annotation was clear for you, my dear peer.